# Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import optuna


from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Data Loading

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
train

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience,blueWins
0,0,0,5,8,6,0,0,14536,17256,0
1,1,1,10,1,5,0,0,14536,17863,0
2,2,0,3,10,2,0,0,17409,17256,0
3,3,1,7,10,8,0,0,19558,18201,0
4,4,0,4,9,4,0,0,17409,17256,0
...,...,...,...,...,...,...,...,...,...,...
7995,9993,1,9,6,14,0,0,18513,18201,1
7996,9994,0,3,10,4,0,0,17381,19797,0
7997,9995,1,9,3,9,2,1,18274,18491,1
7998,9997,0,5,10,6,1,1,18274,18491,1


In [4]:
test

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience
0,9,0,7,6,6,0,0,16961,18201
1,15,0,6,6,6,2,1,18513,18021
2,18,1,6,4,3,0,0,13475,17256
3,23,0,5,4,7,0,0,17409,17256
4,31,0,10,8,9,0,0,18117,18472
...,...,...,...,...,...,...,...,...,...
1995,9971,0,6,3,7,0,0,17409,17256
1996,9980,0,4,4,4,0,0,17409,17256
1997,9983,1,6,3,3,1,1,18513,18201
1998,9996,1,10,9,9,1,1,18513,18201


# Feature

- 各特長量の説明

| ヘッダ名称            | データ型 | 説明                                                       |
|---------------------|-------|----------------------------------------------------------|
| gameId             | int   | ゲームID                                                    |
| blueFirstBlood      | int   | ゲームの最初のキル。青チームが最初のキルを行った場合は1、それ以外の場合は0 |
| blueKills           | int   | 青チームによって殺された敵の数                                     |
| blueDeaths          | int   | 青チームの死亡者数                                            |
| blueAssists         | int   | 青チームのキルアシストの数                                      |
| blueEliteMonsters   | int   | 青チームによって殺されたエリートモンスターの数（ドラゴンとヘラルド）       |
| blueDragons         | int   | 青チームによって殺されたドラゴンの数                               |
| blueTotalGold       | int   | 青チームの得たゴールド合計                                       |
| blueTotalExperience | int   | 青チームの得た経験値合計                                       |
| blueWins            | int   | 目的変数（青チームが勝った場合は1、それ以外の場合は0。）                |


# Feature Engineering

- Add new feature

In [5]:
def create_features(df):
    """
    与えられたデータフレームに対して特徴量エンジニアリングを行う関数。

    Args:
    df (DataFrame): 特徴量エンジニアリングを行うデータフレーム

    Returns:
    DataFrame: 新しい特徴量が追加されたデータフレーム
    """
    # キルとデスの比率
    df['killDeathRatio'] = df['blueKills'] / (df['blueDeaths'] + 1)  # ゼロ除算を防ぐために+1

    # アシストの割合
    df['assistRate'] = df['blueAssists'] / (df['blueKills'] + 1)

    # ヘラルドの討伐数
    df['blueHerald'] = df['blueEliteMonsters'] - df['blueDragons']
    # blueHerald列の負の値を0で置き換え
    df['blueHerald'] = df['blueHerald'].clip(lower=0)

    # エリートモンスターとドラゴンの比率
    df['eliteMonsterDragonRatio'] = df['blueEliteMonsters'] / (df['blueDragons'] + 1)
    # エリートモンスターとヘラルドの比率
    df['eliteMonsterHeraldRatio'] = df['blueEliteMonsters'] / (df['blueHerald'] + 1)

    # ゴールドと経験値の対数変換
    df['logTotalGold'] = np.log1p(df['blueTotalGold'])
    df['logTotalExperience'] = np.log1p(df['blueTotalExperience'])

    # ゴールドと経験値の相互作用
    df['goldXexperience'] = df['blueTotalGold'] * df['blueTotalExperience']

    # 初期キルの影響
    df['firstBloodKillRatio'] = df['blueFirstBlood'] * df['blueKills']
    
    # キル、デス、アシストの合計
    df['totalCombatPoints'] = df['blueKills'] + df['blueDeaths'] + df['blueAssists']

    # キル・デス・アシスト比率
    total_combat_actions = df['blueKills'] + df['blueDeaths'] + df['blueAssists']
    df['killRatio'] = df['blueKills'] / total_combat_actions
    df['deathRatio'] = df['blueDeaths'] / total_combat_actions
    df['assistRatio'] = df['blueAssists'] / total_combat_actions

    # 効率的なリソース利用
    df['goldPerKill'] = df['blueTotalGold'] / (df['blueKills'] + 1)  # ゼロ除算を防ぐために+1
    df['experiencePerKill'] = df['blueTotalExperience'] / (df['blueKills'] + 1)  # ゼロ除算を防ぐために+1

    # エリートモンスターの討伐効率
    df['eliteMonstersPerGold'] = df['blueEliteMonsters'] / df['blueTotalGold']
    df['eliteMonstersPerExperience'] = df['blueEliteMonsters'] / df['blueTotalExperience']

    # チームの総合力
    df['teamStrength'] = df['blueKills'] + df['blueAssists'] + df['blueEliteMonsters']

    return df

- Apply data

In [6]:
train = create_features(train)
test = create_features(test)

# model

- LightGBM

In [7]:


# データの準備
X = train.drop('blueWins', axis=1)
y = train['blueWins']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'binary_error',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', -1, 15),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0)
    }

    # 以前のコードと同じ
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

    verbose_eval = 0
    verbose = lgb.log_evaluation(verbose_eval)
    early_stopping = lgb.early_stopping(stopping_rounds=10, verbose=True)
    model = lgb.train(param, lgb_train, num_boost_round=1000, valid_sets=[lgb_val], callbacks=[early_stopping, verbose])
    
    y_pred = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = (y_pred > 0.5).astype(int)
    accuracy = accuracy_score(y_val, y_pred)

    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# 最適なハイパーパラメータでモデルを再学習
best_params = study.best_params
best_params['objective'] = 'binary'
best_params['metric'] = 'binary_error'
best_params['verbosity'] = -1

lgb_train = lgb.Dataset(X, y)
model = lgb.train(best_params, lgb_train, num_boost_round=study.best_trial.number)

# テストデータに対する予測
y_test_pred = model.predict(test)
y_test_pred_int = (y_test_pred > 0.5).astype(int)

# 予測結果の表示
print(y_test_pred_int)


[I 2024-01-18 11:45:03,019] A new study created in memory with name: no-name-fcb97230-b54e-4502-8d5b-8c0552f7989e
[I 2024-01-18 11:45:03,277] Trial 0 finished with value: 0.755625 and parameters: {'lambda_l1': 6.110575499542893e-05, 'lambda_l2': 3.3622471759407515e-06, 'num_leaves': 128, 'feature_fraction': 0.8022960668924319, 'bagging_fraction': 0.6735461703553447, 'bagging_freq': 3, 'min_child_samples': 27, 'learning_rate': 0.2109175947079513, 'max_depth': 1, 'subsample': 0.21454069041511906, 'colsample_bytree': 0.14187904940603546}. Best is trial 0 with value: 0.755625.


[I 2024-01-18 11:45:03,349] Trial 1 finished with value: 0.755625 and parameters: {'lambda_l1': 3.600120177570044e-07, 'lambda_l2': 1.4335983748287268e-06, 'num_leaves': 249, 'feature_fraction': 0.5285706780923433, 'bagging_fraction': 0.6473028131709921, 'bagging_freq': 6, 'min_child_samples': 78, 'learning_rate': 0.22019424915390415, 'max_depth': 1, 'subsample': 0.2894797505089553, 'colsample_bytree': 0.8560103070812332}. Best is trial 0 with value: 0.755625.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_error: 0.244375
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_error: 0.244375


[I 2024-01-18 11:45:03,384] Trial 2 finished with value: 0.755625 and parameters: {'lambda_l1': 6.750068903032758e-08, 'lambda_l2': 0.03346355647000886, 'num_leaves': 25, 'feature_fraction': 0.5589992666357977, 'bagging_fraction': 0.49855043452685166, 'bagging_freq': 2, 'min_child_samples': 19, 'learning_rate': 0.06587010802795222, 'max_depth': 1, 'subsample': 0.582765009346882, 'colsample_bytree': 0.9464848429014273}. Best is trial 0 with value: 0.755625.
[I 2024-01-18 11:45:03,432] Trial 3 finished with value: 0.784375 and parameters: {'lambda_l1': 9.40277546542025e-05, 'lambda_l2': 8.237076557579097e-08, 'num_leaves': 134, 'feature_fraction': 0.6446375235060591, 'bagging_fraction': 0.7706695861126657, 'bagging_freq': 5, 'min_child_samples': 94, 'learning_rate': 0.01788207101271322, 'max_depth': 5, 'subsample': 0.27417925629846834, 'colsample_bytree': 0.4469622763615255}. Best is trial 3 with value: 0.784375.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_error: 0.244375
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[12]	valid_0's binary_error: 0.215625
Training until validation scores don't improve for 10 rounds


...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	valid_0's binary_error: 0.199375
Training until validation scores don't improve for 10 rounds


[I 2024-01-18 11:45:21,730] Trial 98 finished with value: 0.8075 and parameters: {'lambda_l1': 0.07200361360817625, 'lambda_l2': 0.009163487366992292, 'num_leaves': 248, 'feature_fraction': 0.8390215219559932, 'bagging_fraction': 0.8829386297327909, 'bagging_freq': 6, 'min_child_samples': 42, 'learning_rate': 0.1282162409554457, 'max_depth': 14, 'subsample': 0.7473002800442392, 'colsample_bytree': 0.6195766032540526}. Best is trial 67 with value: 0.815625.


Early stopping, best iteration is:
[19]	valid_0's binary_error: 0.1925
Training until validation scores don't improve for 10 rounds


[I 2024-01-18 11:45:21,999] Trial 99 finished with value: 0.803125 and parameters: {'lambda_l1': 0.12544462967887668, 'lambda_l2': 0.0023306711238854265, 'num_leaves': 233, 'feature_fraction': 0.8731767743780708, 'bagging_fraction': 0.9275802498664731, 'bagging_freq': 7, 'min_child_samples': 46, 'learning_rate': 0.10851758093834823, 'max_depth': 12, 'subsample': 0.7050844571426375, 'colsample_bytree': 0.6671922711833967}. Best is trial 67 with value: 0.815625.


Early stopping, best iteration is:
[21]	valid_0's binary_error: 0.196875
[1 1 0 ... 1 1 0]


- testにy_test_pred結合

In [8]:
output = '../data/output/submit_cv_oputuna_add_feature_addparams.csv'

In [9]:
submit = pd.DataFrame({
    '0': test['gameId'],
    '1': y_test_pred_int
})

# カラムヘッダー消去して出力
# ヘッダーなしでCSVファイルとして保存
submit.to_csv(output, header=False, index=False)

- 出力csvチェック

In [10]:
check = pd.read_csv(output)

In [11]:
check

,9,1
0,15,1
1,18,0
2,23,0
3,31,0
4,32,1
...,...,...
1994,9971,0
1995,9980,0
1996,9983,1
1997,9996,1
